**url**

https://news.abbvie.com/news/allergan-press-releases/

In [9]:
import pandas as pd,requests,bs4,re,time,io,pdftotext
from selenium.webdriver.common.by import By
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [10]:
SITE_NAME='Abbvie'

DOMAIN = "https://news.abbvie.com"

SITE_LINK="https://news.abbvie.com/news/allergan-press-releases/"

In [11]:
def remove_esc_chars(text):
    return text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

In [12]:
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [13]:
article_list = []
pagination = 2020
last_available_data = 2018
last_available_data = 2020
while True:
    for i in range(1,131,10):
        url = f"https://news.abbvie.com/news/allergan-press-releases/?start_row={i}&archiveYear={pagination}"
        soup = parse_webpage_bs(url)
        if soup != None:
            elements = soup.findAll('div',{'class' :'col-sm-12 pad-bottom10'})
            if len(elements) == 0:
                print("\n---------------- Reached at the end of the page ----------------\n")
                break
                
            for element in elements:
                #title,,link,thumbnail and author
                title = element.find('h4',{'class' :'media-heading'}).text.strip()
                link = DOMAIN + element.find('h4',{'class' :'media-heading'}).find('a')['href'].strip()
                thumbnail = "https://www.abbvie.com/content/experience-fragments/abbvie-com/us/en/header/master/_jcr_content/root/globalnavigation/logo.coreimg.svg/1648149043863/logo.svg"
                author = SITE_NAME

                #published_date
                published_date = element.find('div',{'class' :'rel-date'}).text.strip()
                
                #text
                g = Goose()
                article_extract = g.extract(url=link)
                article = remove_esc_chars(article_extract.cleaned_text)
                meta_data = remove_esc_chars(article_extract.meta_description)
                whole_data = meta_data+article
                text = whole_data.strip()
                
                article = (published_date.strip(),title.strip(),text.strip(),link.strip(),thumbnail.strip(),author.strip())
                article_list.append(article)
                print(published_date,title)

    if pagination <= last_available_data :
        break
    else:
        pagination -=1

May 08, 2020 AbbVie Inc. Completes Acquisition of Allergan plc
May 07, 2020 The Allergan Foundation Doubles its COVID-19 Response Donations to $4.0 Million
May 06, 2020 Announcement relating to the outcome of the Court Sanction Hearing
Apr 28, 2020 Announcement relating to the voluntary delisting of Allergan Notes
Apr 02, 2020 China Gives Greenlight to Allergan's XEN® Registration for the Surgical Management of Refractory Glaucoma
Mar 31, 2020 Allergan Earns ENERGY STAR® Partner of the Year - Sustained Excellence Award for the Seventh Consecutive Year
Mar 30, 2020 The Allergan Foundation Donates $2.0 Million to Community Organizations on Front-Line of COVID-19 Response
Mar 18, 2020 Allergan Declares Second Quarter 2020 Cash Dividend of $0.74 Per Ordinary Share
Mar 10, 2020 Announcement under the Irish Takeover Rules: Relevant Securities in Issue - March 10, 2020
Mar 05, 2020 Allergan Receives FDA Approval for DURYSTA™ (bimatoprost implant) the First and Only Intracameral Biodegradable 

In [14]:
temp_df = pd.DataFrame(article_list,columns=['date','title','article','url','thumbnail','author'])
temp_df.head()

,date,title,article,url,thumbnail,author
0,"May 08, 2020",AbbVie Inc. Completes Acquisition of Allergan plc,"NOT FOR RELEASE, PUBLICATION OR DISTRIBUTION, ...",https://news.abbvie.com/news/press-releases/ne...,https://www.abbvie.com/content/experience-frag...,Abbvie
1,"May 07, 2020",The Allergan Foundation Doubles its COVID-19 R...,", /PRNewswire/ -- (NYSE: AGN) today announced ...",https://news.abbvie.com/news/press-releases/ne...,https://www.abbvie.com/content/experience-frag...,Abbvie
2,"May 06, 2020",Announcement relating to the outcome of the Co...,"NOT FOR RELEASE, PUBLICATION OR DISTRIBUTION, ...",https://news.abbvie.com/news/press-releases/ne...,https://www.abbvie.com/content/experience-frag...,Abbvie
3,"Apr 28, 2020",Announcement relating to the voluntary delisti...,"NOT FOR RELEASE, PUBLICATION OR DISTRIBUTION, ...",https://news.abbvie.com/news/press-releases/ne...,https://www.abbvie.com/content/experience-frag...,Abbvie
4,"Apr 02, 2020",China Gives Greenlight to Allergan's XEN® Regi...,-- XEN® Becomes the First Medical Product Appr...,https://news.abbvie.com/news/press-releases/ne...,https://www.abbvie.com/content/experience-frag...,Abbvie


In [15]:
#to csv
temp_df.to_csv(f'{SITE_NAME} news.csv',index = False)

#to json
temp_df.to_json(f'{SITE_NAME} news.json')